In [ ]:
import numpy as np

import plotly.graph_objs as go

from scipy.special import legendre, roots_legendre
from sympy.integrals.quadrature import gauss_legendre

# Un programme d'intégration adaptatif

## Construction d'une quadrature adaptative

L'objectif de cette section supplémentaire est de voir comment on peut construire une quadrature adaptative basée sur la quadrature de Gauss.
De manière plus précise,  pour une fonction $f:[a, b] \rightarrow \mathbb{R}$ donnée, on cherche à calculer 
la valeur de $\int_{a}^{b} f(x) {\mathrm d}x$ à une précision relative de $\mathrm{rTol}$. Si l'on fixe la formule de quadrature (par exemple la formule de Gauss d'ordre 30 avec $s=15$ ), 
il faut trouver une division $\Delta=\left\{a=x_{0}<x_{1}<\ldots<x_{N}=b\right\}$ de l'intervalle $(a, b)$ afin que l'approximation numérique via la formule composée basée sur  la formule de quadrature de Gauss à l'ordre 30 à 15 étages, 
$I_{\Delta}^{15}$ satisfasse :

$$
\left|I_{\Delta}^{15}-\int_{a}^{b} f(x) {\mathrm d}x\right| \leq \mathrm{rTol}  \int_{a}^{b}|f(x)| {\mathrm d}x
$$

Pour une fonction $f$ ne changeant pas de signe sur $(a, b)$, la condition précédente signifie que l'erreur relative au sens de la norme infinie est bornée par $\mathrm{rTol}$.
Pour construire cette quadrature adaptative, nous sommes  confrontés à deux problèmes~:
 * le choix de la division pour que  l'inégalité précédente soit satisfaite;
 * construire une estimation de l'erreur $I_{\Delta}-\int_{a}^{b} f(x) {\mathrm d}x$.

### Détermination de la division

Pour un sous-intervalle $\left(x_{0}, x_{0}+h\right)$ de $(a, b),$ on sait calculer les valeurs:

$$
\operatorname{res}\left(x_{0}, x_{0}+h\right) &=h \sum_{i=0}^{s-1} b_{i} f\left(x_{0}+c_{i} h\right) \\
\text {resabs}\left(x_{0}, x_{0}+h\right) &=h \sum_{i=0}^{s-1} b_{i}\left|f\left(x_{0}+c_{i} h\right)\right|
$$

Supposons, pour le moment, qu'on connaisse aussi une estimation de l'erreur

$$
\operatorname{err}\left(x_{0}, x_{0}+h\right) \approx \operatorname{res}\left(x_{0}, x_{0}+h\right)-\int_{x_{0}}^{x_{0}+h} f(x) {\mathrm d}x
$$

L'algorithme pour trouver une division convenable est le suivant:

on calcule $\operatorname{res}(a, b), \operatorname{resabs}(a, b)$ et $\operatorname{err}(a, b) .$ Si

$$
|\operatorname{err}(a, b)| \leq \mathrm{rTol}  \operatorname{resabs}(a, b)
$$

on accepte $\operatorname{res}(a, b)$ comme approximation de $\int_{a}^{b} f(x) {\mathrm d}x$ et on arrête le calcul; sinon

on subdivise $(a, b)$ en deux sous-intervalles $I_{1}=(a,(a+b) / 2)$ et $I_{2}=((a+b) / 2, b)$ et on calcule $\operatorname{res}\left(I_{1}\right),$ resabs $\left(I_{1}\right), \operatorname{err}\left(I_{1}\right)$ et $\operatorname{res}\left(I_{2}\right),$ resabs $\left(I_{2}\right), \operatorname{err}\left(I_{2}\right) .$ On pose $N=2$ et on
regarde si:

$$
\sum_{j=1}^{N}\left|\operatorname{err}\left(I_{j}\right)\right| \leq \operatorname{\mathrm{rTol}} \left(\sum_{j=1}^{N} \operatorname{resabs}\left(I_{j}\right)\right).
$$

Si l'inégalité précédente est vérifiée, on accepte $\operatorname{res}\left(I_{1}\right)+\operatorname{res}\left(I_{2}\right)$ comme approximation de $\int_{a}^{b} f(x) {\mathrm d}x ;$ sinon

on pose $N:=N+1$ et on subdivise l'intervalle où l'erreur est maximale (disons $I_{k}$ ) en deux sous-intervalles équidistants qu'on denote par $I_{k}$ et $I_{N+1}$. Ensuite, on calcule res, resabs et err pour ces deux intervalles. Si l'inégalité précédente est vérifié, on arrête le calcul et on accepte

$$
\sum_{j=1}^{N} \operatorname{res}\left(I_{j}\right) \approx \int_{a}^{b} f(x) {\mathrm d}x
$$

comme approximation de l'intégrale; sinon on répète la partie (iii) de cet algorithme.

### Estimation d'erreur
 
 Malheureusement, les formules pour l'erreur obtenues au début du cours ne sont pas très utiles  pour ce nouvel objectif car  on ne connait que rarement la $p^{\text {ème }}$ dérivée de la fonction $f(x)$, en particulier 
 quand $p=30$!
L'idée est d'appliquer une autre formule de quadrature d'ordre plus faible $\left(\widehat{b}_{i}, \widehat{c}_{i}\right) \hat{s}_{i=1}$ et d'utiliser la différence de deux approximations numériques comme estimation de l'erreur du  résultat
le moins précis. 
Pour que le travail supplémentaire soit très faible, on suppose $\widehat{s} \leq s$ et on reprend les mêmes évaluations de $f$, c'est-à-dire que l'on suppose $\widehat{c}_{i}=c_{i}$ pour tous $i$. 

Une telle formule de quadrature s'appelle formule emboitée, si pour au moins un indice $i$ on a $\hat{b}_{i} \neq b_{i}$.
Si $\left(b_{i}, c_{i}\right)_{i=0}^{s-1}$ est une formule de quadrature d'ordre $p \geq s,$ l'ordre d'une formule emboitée est $\widehat{p} \leq s-1 .$ Ce résultat découle du fait que, pour une formule de quadrature d'ordre $\geq s,$ les poids $b_{i}$ sont uniquement déterminés par ses n\oe uds $c_{i}$.

Pour la formule de Gauss $(s=15, p=30),$ on obtient une formule emboitée $\left(\widehat{b}_{i}, c_{i}\right)_{i=0}^{s-1}$ d'ordre 14 en enlevant le point milieu $c_{8}=1 / 2$, c.-à-d., on pose $\hat{b}_{8}=0$. 
L'expression calculable:

$$
\mathrm{err}_1:=h \sum_{i=0}^{s-1} b_{i} f\left(x_{0}+c_{i} h\right)-h \sum_{i=0}^{s-1} \widehat{b}_{i} f\left(x_{0}+c_{i} h\right) \approx C_{1} h^{15}
$$

est une approximation de l'erreur de la formule emboitée, car

$$
\left(h \sum_{i=1}^{s} b_{i} f\left(x_{0}+c_{i} h\right)-\int_{x_{0}}^{x_{0}+h} f(x) d x\right)+\left(\int_{x_{0}}^{x_{0}+h} f(x) d x-h \sum_{i=1}^{s} \widehat{b}_{i} f\left(x_{0}+c_{i} h\right)\right)
$$

avec une estimation sur le premier terme en $C h^{31}+\mathcal{O}\left(h^{32}\right)$ et sur le second en $C h^{15}+\mathcal{O}\left(h^{16}\right)$.

Afin procéder à l'adaptation, on considère encore une deuxième formule emboitée qui a pour nœuds $\left\{c_{2}, c_{4}, c_{6}, c_{10}, c_{12}, c_{14}\right\}$ et un ordre 6.
Les poids de cette formule de quadrature sont notés $\hat{b}_{i}$ et on définit :

$$
\mathrm{err}_2:=h \sum_{i=0}^{s-1} b_{i} f\left(x_{0}+c_{i} h\right)-h \sum_{i=0}^{s-1} \widehat{\widehat{b}}_{i} f\left(x_{0}+c_{i} h\right) \approx C_{2} h^{7}.
$$

Il y a plusieurs possibilités pour définir $\operatorname{err}\left(x_{0}, x_{0}+h\right)$:
* $\operatorname{err}\left(x_{0}, x_{0}+h\right):= \mathrm{err}_1$; cette estimation est trop pessimiste. En général, la formule de Gauss donne un résultat largement meilleur que la formule emboitée d'ordre $14$.
* pour avoir une estimation relativement optimale, on a choisi l'approximation

$$
\operatorname{err}\left(x_{0}, x_{0}+h\right):=\mathrm{err}_1 \left(\frac{\mathrm{err}_1}{\mathrm{err}_2}\right)^{2}, \quad\left(\approx h^{15} \left(\frac{h^{15}}{h^{7}}\right)^{2} \approx h^{31}\right),
$$

ce qui donne de très bons résultats.

**Abscisses de la formule de Gauss à 15 points et ses formules emboitées à 14 et 6 points :**

In [ ]:
p = legendre(15)

p30 = (np.sort(p.r)+1.)/2
p15 = np.delete(p30, 7)
p06 = p15[[1, 3, 5, 8, 10, 12]]

fig = go.Figure()
fig.add_trace(go.Scatter(x=p30, y=np.zeros(p30.size), mode='markers', name='Formule de Gauss (ordre 30)'))
fig.add_trace(go.Scatter(x=p15, y=np.zeros(p15.size)+0.2, mode='markers', name='Formule emboitée (ordre 15)'))
fig.add_trace(go.Scatter(x=p06, y=np.zeros(p06.size)+0.4, mode='markers', name='Formule emboitée (ordre 6)'))
fig.update_yaxes(range=[-0.1, 0.5])
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.85))
fig.show()

**Poids de la formule de Gauss à 15 points (sympy - précision arbitraire) :**

In [ ]:
# 15 points de Gauss, avec une précision de 20 chiffres significatifs
x, w = gauss_legendre(15, 20)
print("Racine du polynome de Legendre sur [-1,1]:")
print(x)
print("Poids :")
print(w)

# On ramène sur [0,1]
xx = (np.array(x,dtype = "float"))/2+1./2.
ww = np.array(w,dtype = "float")/2.

fig = go.Figure()
fig.add_trace(go.Scatter(x=xx, y=ww, mode='lines+markers', line_dash='dash', name="Gauss-Legendre - 15pts", showlegend=True))
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.25))
fig.show()

In [ ]:
def integral_step(f, xmin, xmax, r, w):
    h = xmax-xmin
    return h*np.sum(w * f(xmin+h*r))

def gauss(f, xmin, xmax, s, n):
    r, w = roots_legendre(s)
    r = 0.5*(r+1)
    w = 0.5*w
    x = np.linspace(xmin, xmax, n+1)
    res = 0.
    for i in range(n):
        res += integral_step(f, x[i], x[i+1], r, w)
    return res

def adapt(f, xmin, xmax, res_ref, tol=1.e-10, n_max=50, verbose=False):

    # calcule des abscisses et des points de la méthode de Gauss-Legendre ordre 15 à 30 étages
    r_30, w_30 = roots_legendre(15)
    r_30 = 0.5*(r_30+1.)
    w_30 = 0.5*w_30
    
    # calcule des abscisses et des points de la méthode emboitée d'ordre 6 à 6 étages
    r_06 = r_30[[1,3,5,9,11,13]]
    a = np.array([r_06**0, r_06**1, r_06**2, r_06**3, r_06**4, r_06**5])
    b = 1./np.arange(1,7)
    w_06 = np.linalg.solve(a, b)
    
    # calcule des abscisses et des points de la méthode emboitée d'ordre 6 à 6 étages
    r_14 = r_30[[0,1,2,3,4,5,6,8,9,10,11,12,13,14]]
    a = np.array([r_14**0, r_14**1, r_14**2, r_14**3,  r_14**4,  r_14**5,  r_14**6, 
                  r_14**7, r_14**8, r_14**9, r_14**10, r_14**11, r_14**12, r_14**13])
    b = 1./np.arange(1,15)
    w_14 = np.linalg.solve(a, b)
    
    def adapt_step(xmin, xmax):
        
        h = xmax-xmin
        f_30 = f(xmin+h*r_30)
        
        res_30 =  h*np.sum(w_30 * f_30)
        res_14 =  h*np.sum(w_14 * f_30[[0,1,2,3,4,5,6,8,9,10,11,12,13,14]])
        res_06 =  h*np.sum(w_06 * f_30[[1,3,5,9,11,13]])
 
        err_1 = res_30 - res_14 
        err_2 = res_30 - res_06 
        err_est = err_1 * (err_1/err_2)**2
        
        return res_30, err_est 

    # initialization 
    res, err_est = adapt_step(xmin, xmax)
    #print(res,res_ref) 
    err_previous = abs(res - res_ref)
    #print(err_previous)
    
    if (np.abs(err_est) < tol*np.abs(res)):
        return res

    n = 2
 
    err_est_array = np.zeros(n_max)
    res_array = np.zeros(n_max)
  
    a = xmin
    b = xmax

    div = []
    div.append((a, (a+b)/2))
    div.append(((a+b)/2, b))
    
    div_per_ite = []
    div_per_ite.append(div.copy())
        
    ierrmax = 0
    
    while (n <= n_max):
                        
        res_array[ierrmax], err_est_array[ierrmax] = adapt_step(a, (a+b)/2)
        res_array[n-1], err_est_array[n-1] = adapt_step((a+b)/2, b)
                
        absres = np.sum(np.abs(res_array))
        res = np.sum(res_array)
        err_est = np.sum(np.abs(err_est_array))
        err     = np.abs(res - res_ref)
 
        if (verbose):
            print(f"It={n-1:2d}: res={res:14.12f}, |err|={err:16.10e}, |err est.|={err_est:16.10e}") 
            print(f"                            |err est.|/absres={err_est/absres:16.10e}, errN/errN-1 ={err/err_previous:6.5f}") 
        
        if (err_est < tol*absres):
            break
        
        err_previous = err
        
        ierrmax = np.argmax(np.abs(err_est_array)) 
        a, b = div[ierrmax]
        div[ierrmax] = (a, (a+b)/2)
        div.append(((a+b)/2, b))
            
        div_per_ite.append(div.copy()) 
        
        n += 1
    
    return res, div_per_ite

## Premier exemple

On considère la fonction :

$$ f(x) = 2 + \sin(3 \, cos(0.002(x-40)^2)) \quad \text{sur }[10,110] $$

In [ ]:
def f(x):
    return 2 + np.sin(3*np.cos(0.002*(x-40)**2))

x = np.linspace(10, 110, 2000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=f(x)))

fig.show()

**Application de l'algorithme d'adaptation :**

In [ ]:
res_ref = gauss(f, 10, 110, 30, 10000)

print("Algorithme adaptatif :")
res, div_per_ite = adapt(f, 10., 110., res_ref, tol=1.e-10, n_max=50, verbose=True)

# Soulignons que N = nombre d'itération + 1 = nombre d'intervalles
print(f"\nNb d'itération (subdivision par 2) : {len(div_per_ite)} pour satisfaire |err estimée|/absres < tol  et {len(div_per_ite)+1} Intervalles")

In [ ]:
x = np.linspace(10, 110, 2000)

fig = go.Figure()
#fig.add_trace(go.Scatter(x=x, y=f(x), line_color='grey', showlegend=False))

for ite in range(1, len(div_per_ite)+1):
    for div in div_per_ite[ite-1]:
        xdiv = np.linspace(div[0], div[1], 100)
        fig.add_trace(go.Scatter(visible=False, x=xdiv, y=f(xdiv), fill='tozeroy', showlegend=False))

        
# Make plot visible for ite=1
for i in range(len(div_per_ite[0])): fig.data[i].visible = True
    
# Create and add slider
ivis = 0
steps = []
for ite in range(1, len(div_per_ite)+1):
    step = dict(method="update", label = f" {ite}", args=[{"visible": [False] * len(fig.data)}])
    for idiv, div in enumerate(div_per_ite[ite-1]):
        #print(idiv, ivis)
        step["args"][0]["visible"][ivis] = True
        ivis = ivis+1
    steps.append(step)

sliders = [dict(currentvalue={"prefix": "ite: "}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_xaxes(tickmode = 'array', tickvals = [div[0] for div in div_per_ite[-1]] + [110])

fig.show()

## Deuxième exemple

On considère la fonction :

$$ g(x) = \sqrt{x} \, \log{x} \quad \text{sur }]0,1] $$

In [ ]:
def g(x):
    return  np.sqrt(x)*np.log(x)

x = np.linspace(1.e-10, 1, 10000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=g(x)))

fig.show()

**Application de l'algorithme d'adaptation:**

In [ ]:
res_ref =  -4/9

print("Algorithme adaptatif :")
res, div_per_ite = adapt(g, 1.e-20, 1, res_ref, tol=1.e-13, n_max=50, verbose=True)
print(f"\nNb d'itération pour satisfaire |err estimée|/absres < tol : {len(div_per_ite)}")

In [ ]:
x = np.linspace(1.e-10, 1, 10000)

fig = go.Figure()

for ite in range(1, len(div_per_ite)+1):
    for div in div_per_ite[ite-1]:
        xdiv = np.linspace(div[0], div[1], 100)
        fig.add_trace(go.Scatter(visible=False, x=xdiv, y=g(xdiv), fill='tozeroy', showlegend=False))

# Make plot visible for ite=1
for i in range(len(div_per_ite[0])): fig.data[i].visible = True
    
# Create and add slider
ivis = 0
steps = []
for ite in range(1, len(div_per_ite)+1):
    step = dict(method="update", label = f" {ite}", args=[{"visible": [False] * len(fig.data)}])
    for idiv, div in enumerate(div_per_ite[ite-1]):
        #print(idiv, ivis)
        step["args"][0]["visible"][ivis] = True
        ivis = ivis+1
    steps.append(step)

sliders = [dict(currentvalue={"prefix": "ite: "}, steps=steps)]

fig.update_layout(sliders=sliders)

fig.show()